<a href="https://colab.research.google.com/github/anishthetechie/Market-Making/blob/main/MarketMaking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import random
from collections import deque
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers

class OrderBook:
    def __init__(self):
        self.bids = []  # List of (price, quantity) tuples
        self.asks = []  # List of (price, quantity) tuples
        self.mid_price = 100  # Initial mid-price
        self.inventory = 0  # Market maker's inventory

    def update_price(self):
        self.mid_price += np.random.normal(0, 0.5)  # Simulated price movement

    def place_order(self, price, quantity, order_type='bid'):
        if order_type == 'bid':
            self.bids.append((price, quantity))
            self.bids.sort(reverse=True)
        else:
            self.asks.append((price, quantity))
            self.asks.sort()

    def match_orders(self):
        while self.bids and self.asks and self.bids[0][0] >= self.asks[0][0]:
            bid_price, bid_qty = self.bids.pop(0)
            ask_price, ask_qty = self.asks.pop(0)
            trade_qty = min(bid_qty, ask_qty)
            self.inventory += trade_qty  # Adjust inventory
            if bid_qty > trade_qty:
                self.bids.insert(0, (bid_price, bid_qty - trade_qty))
            if ask_qty > trade_qty:
                self.asks.insert(0, (ask_price, ask_qty - trade_qty))

class MarketMakerRL:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # Discount factor
        self.epsilon = 1.0  # Exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = models.Sequential([
            layers.Dense(24, activation='relu', input_dim=self.state_size),
            layers.Dense(24, activation='relu'),
            layers.Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=optimizers.Adam(self.learning_rate))
        return model

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return  # Skip training if not enough data
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target += self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

if __name__ == "__main__":
    env = OrderBook()
    state_size = 4  # Example: mid-price, spread, inventory, volatility
    action_size = 3  # Example: widen spread, tighten spread, hold
    agent = MarketMakerRL(state_size, action_size)

    episodes = 10
    max_steps_per_episode = 200  # Avoid infinite loops

    for e in range(episodes):
        env = OrderBook()
        env.update_price()
        state = np.reshape([env.mid_price, 1, env.inventory, 0.5], [1, state_size])
        total_reward = 0

        for step in range(max_steps_per_episode):
            action = agent.act(state)
            env.update_price()
            env.match_orders()

            reward = np.random.normal(1, 0.1) - abs(env.inventory) * 0.01  # Simulated PnL with inventory penalty
            next_state = np.reshape([env.mid_price, 1, env.inventory, 0.5], [1, state_size])
            agent.remember(state, action, reward, next_state, False)
            state = next_state
            total_reward += reward

            if abs(env.inventory) > 50:  # Stop episode if inventory is too high
                break

        agent.replay(32)
        print(f"Episode {e+1}/{episodes}, Total Reward: {total_reward:.2f}")

    print("Training complete!")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━